In [2]:
import time
import os

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision
from torchvision import transforms

import PIL
from PIL import Image
import matplotlib.pyplot as plt
from collections import OrderedDict

from utility import *
from vgg_network import *

In [3]:
# Get network
vgg = VGG()
vgg.load_state_dict(torch.load('../Models/vgg_conv.pth'))
for param in vgg.parameters():
    param.requires_grad = False
if torch.cuda.is_available():
    vgg.cuda()

# Get image paths
# Style with serifs
style_path1 = '../input/serif_fonts/'
style_name1 = 'ichi_serif.bmp'
style_dir1 = style_path1 + style_name1

# Style that lacks serifs
style_path2 = '../input/serif_fonts/'
style_name2 = 'ichi.bmp'
style_dir2 = style_path2 + style_name2

# Content 
content_path = '../input/serif_fonts/'
content_name = 'ni.bmp'
content_dir = content_path + content_name


# Get output path
output_path = '../output_from_pytorch/type8/'
try:
    os.mkdir(output_path)
except:
    pass
output_path = output_path + content_name[:-4] + '_' + style_name1[:-4] + '_' + style_name2[:-4] 

# Load images
style_image1, style_image2, content_image, opt_img = load_images(style_dir1, style_dir2, content_dir)

In [9]:
# Define layers, loss functions, weights and compute optimization targets

# Style layers
style_layers = ['r11','r21','r31','r41','r51'] 
style_weights = [1e3/n**2 for n in [64,128,256,512,512]]
# Content layers
content_layers = ['r12','r22','r32','r42','r52']
content_weights = [0,0,0,1e5,0]

loss_layers = style_layers + content_layers
loss_functions = [GramMSELoss()] * len(style_layers) + [nn.MSELoss()] * len(content_layers)
if torch.cuda.is_available():
    loss_functions = [loss_fn.cuda() for loss_fn in loss_functions]
weights = style_weights + content_weights

In [10]:
# Compute optimization targets
### Style targets 
# Gram matrices of 1st style
style_targets1 = [GramMatrix()(A).detach() for A in vgg(style_image1, style_layers)]
# Gram matrices of 2nd style
style_targets2 = [GramMatrix()(A).detach() for A in vgg(style_image2, style_layers)]
# Feature responces of 1st style
style_content1 = [A.detach() for A in vgg(style_image1, content_layers)]
# Feature responces of 2nd style
style_content2 = [A.detach() for A in vgg(style_image2, content_layers)]
style_targets = []
for i in range(len(style_targets1)):
    style_targets.append(0.5*(style_targets2[i] + style_targets1[i]))
style_content = []
for i in range(len(style_content1)):
    style_content.append(0.5*(style_content2[i] + style_content1[i]))
### Content targets
# Feature responces of content
content_targets = [A.detach() for A in vgg(content_image, content_layers)]
# Gram matrices of content
content_styles  = [GramMatrix()(A).detach() for A in vgg(content_image, style_layers)]

In [11]:
# Run style transfer
try:
    os.mkdir(output_path)
except:
    pass

max_iter = 5000
show_iter = 100
optimizer = optim.LBFGS([opt_img]);
n_iter=[0]
loss_list = []

while n_iter[0] <= max_iter:

    def closure():
        optimizer.zero_grad()
        out = vgg(opt_img, loss_layers)
        content_layer_losses = []
        style_layer_losses  = []
        
        opt_style = []
        opt_content = []
        for i, A in enumerate(out):
            if i < len(style_targets):
                opt_style.append(GramMatrix()(A))
            else:
                opt_content.append(A)

        results_style = []
        for i in range(len(content_styles)):
            results_style.append(0.5*(content_styles[i] + opt_style[i]))
        results_content = []
        for i in range(len(content_targets)):
            results_content.append(0.5*(content_targets[i] + opt_content[i]))
        
        for i in range(len(content_styles)):
            style_layer_losses.append(style_weights[i]*(nn.MSELoss()(results_style[i], style_targets[i])))
        for i in range(len(content_targets)):
            content_layer_losses.append(content_weights[i]*(nn.MSELoss()(results_content[i], style_content[i])))
        
        layer_losses = content_layer_losses + style_layer_losses

        content_loss = sum(content_layer_losses)
        style_loss   = sum(style_layer_losses)

        loss = sum(layer_losses)
        loss.backward()
        loss_list.append(loss)

        n_iter[0]+=1
        #print loss
        if n_iter[0]%show_iter == 0:
            print('Iteration: {} \nContent loss: {} \nStyle loss  : {} \nTotal loss  : {}'
            .format(n_iter[0], content_loss.item(), style_loss.item(), loss.item()))

            # Save loss graph
            plt.plot(loss_list)
            plt.savefig(output_path + '/loss_graph.jpg')
            plt.close()
            # Save optimized image
            out_img = postp(opt_img.data[0].cpu().squeeze())
            out_img = PIL.ImageOps.invert(out_img)
            out_img.save(output_path + '/{}.bmp'.format(n_iter[0]))

        return loss
    
    optimizer.step(closure)

# Save sum images
save_images(content_image, opt_img, style_image1, style_image2, output_path, n_iter)

Iteration: 100 
Content loss: 11775476736.0 
Style loss  : 550368832.0 
Total loss  : 12325846016.0
Iteration: 200 
Content loss: 11774223360.0 
Style loss  : 550329216.0 
Total loss  : 12324553728.0
Iteration: 300 
Content loss: 11773563904.0 
Style loss  : 549787584.0 
Total loss  : 12323350528.0
Iteration: 400 
Content loss: 11772583936.0 
Style loss  : 549360256.0 
Total loss  : 12321944576.0
Iteration: 500 
Content loss: 11770596352.0 
Style loss  : 549346368.0 
Total loss  : 12319943680.0
Iteration: 600 
Content loss: 11768114176.0 
Style loss  : 549968896.0 
Total loss  : 12318083072.0
Iteration: 700 
Content loss: 11766009856.0 
Style loss  : 550334976.0 
Total loss  : 12316345344.0
Iteration: 800 
Content loss: 11764187136.0 
Style loss  : 550054720.0 
Total loss  : 12314242048.0
Iteration: 900 
Content loss: 11762302976.0 
Style loss  : 550069056.0 
Total loss  : 12312371200.0
Iteration: 1000 
Content loss: 11760103424.0 
Style loss  : 550318016.0 
Total loss  : 12310420480.0